In [1]:
import pdb
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import json
from flatten_json import flatten
import numpy as np
from tqdm import tqdm
import pickle
import sys
from statsbombpy import sb
from multiprocessing import Pool

from bs4 import BeautifulSoup

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}

In [3]:
import socket
import requests.packages.urllib3.util.connection as urllib3_cn
    
def allowed_gai_family():
    """
     https://github.com/shazow/urllib3/blob/master/urllib3/util/connection.py
    """
    return socket.AF_INET

urllib3_cn.allowed_gai_family = allowed_gai_family

In [4]:
def get_page(l):
    passed = 0
    while passed == 0:
        try:
            page = requests.get(l, headers=headers)
            #print(page)
            if page.status_code == 429:
                print("retrying after ", int(page.headers["Retry-After"]))
                time.sleep(int(page.headers["Retry-After"]))
            else:
                passed = 1
        except:
            passed = 0
            print("trying to download ", l)
            time.sleep(5)
            
    return page

# ADP

In [5]:
pg = get_page("https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php?year=2021")
a = BeautifulSoup(pg.content)

In [6]:
a.find("tbody").find_all("tr")[0].find_all("a")[-1]["class"]

['fp-player-link', 'fp-id-16393']

In [7]:
def get_adp(year_start, year_end, bl):
    years = np.arange(year_start, year_end+1)
    
    dfs = []
    for y in tqdm(years):
        page = get_page(bl+str(y))
        
        df = pd.read_html(page.content)[0]
        try:
            cols1 = df.columns.droplevel(1)
            cols0 = df.columns.droplevel(0)
            
            new_c = []
            for i in range(0, len(cols1), 1):
                if "Unnamed" in cols1[i]:
                    new_c.append(cols0[i])
                else:
                    new_c.append(cols0[i]+"_"+cols1[i])
            df.columns = new_c
        except:
            pass
        df["Season"] = [y]*len(df)
        
        code = BeautifulSoup(page.content)
        trs = code.find("tbody").find_all("tr")
        
        ids = []
        for tr in trs:
            #print(tr)
            nid = int(tr.find_all("a")[-1]["class"][-1].split("-")[-1])
            #print(nid)
            ids.append(nid)
        
        
        df["pid"] = ids
        dfs.append(df)
    
    return dfs

In [8]:
adps = get_adp(2018, 2023, "https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php?year=")

100%|██████████| 6/6 [00:09<00:00,  1.54s/it]


# RBS

In [10]:
#adps[-2]

In [11]:
rbs = get_adp(2015, 2023, "https://www.fantasypros.com/nfl/stats/rb.php?scoring=HALF&range=full&year=")

100%|██████████| 9/9 [00:18<00:00,  2.09s/it]


In [12]:
wrs = get_adp(2015, 2023, "https://www.fantasypros.com/nfl/stats/wr.php?scoring=HALF&range=full&year=")

100%|██████████| 9/9 [00:14<00:00,  1.59s/it]


In [13]:
tes = get_adp(2015, 2023, "https://www.fantasypros.com/nfl/stats/te.php?scoring=HALF&range=full&year=")

100%|██████████| 9/9 [00:15<00:00,  1.72s/it]


In [14]:
qbs = get_adp(2015, 2023, "https://www.fantasypros.com/nfl/stats/qb.php?scoring=HALF&range=full&year=")

100%|██████████| 9/9 [00:15<00:00,  1.74s/it]


In [15]:
line = pd.concat(rbs+wrs+tes)

qb = pd.concat(qbs)

In [16]:
adp = pd.concat(adps)

In [17]:
line_comp = line.merge(adp, how="left", on=["pid", "Season"])

In [18]:
qb_comp = qb.merge(adp, how="left", on=["pid", "Season"])

In [19]:
rbs_comp = pd.concat(rbs).merge(adp, how="left", on=["pid", "Season"])
wrs_comp = pd.concat(wrs).merge(adp, how="left", on=["pid", "Season"])
tes_comp = pd.concat(tes).merge(adp, how="left", on=["pid", "Season"])

In [20]:
rbs_comp.to_csv("raw data/rb_raw.csv", index=False)
wrs_comp.to_csv("raw data/wr_raw.csv", index=False)
tes_comp.to_csv("raw data/te_raw.csv", index=False)

In [21]:
qb_comp.to_csv("raw data/qb_raw.csv", index=False)
line_comp.to_csv("raw data/line_raw.csv", index=False)

# PROJS

In [152]:
proj_rb = pd.concat(get_adp(2020, 2023, "https://www.fantasypros.com/nfl/projections/rb.php?week=draft&scoring=HALF&range=full&year="))

100%|██████████| 4/4 [00:04<00:00,  1.19s/it]


In [153]:
proj_wr = pd.concat(get_adp(2020, 2023, "https://www.fantasypros.com/nfl/projections/wr.php?week=draft&scoring=HALF&range=full&year="))

100%|██████████| 4/4 [00:03<00:00,  1.15it/s]


In [154]:
proj_te = pd.concat(get_adp(2020, 2023, "https://www.fantasypros.com/nfl/projections/te.php?week=draft&scoring=HALF&range=full&year="))

100%|██████████| 4/4 [00:01<00:00,  2.24it/s]


In [177]:
line_proj = pd.concat([proj_rb, proj_wr, proj_te])

In [178]:
line_proj = line_proj.merge(line_comp[["pid", "Season", "FPTS_MISC", "FPTS/G_MISC"]], how="left", on=["pid", "Season"], suffixes = ["_proj", "_real"])

In [179]:
line_proj.to_csv("line_proj.csv", index=False)

In [184]:
rb_proj = proj_rb.merge(rbs_comp[["pid", "Season", "FPTS_MISC", "FPTS/G_MISC", "AVG"]], how="left", on=["pid", "Season"], suffixes = ["_proj", "_real"])

In [185]:
rb_proj.to_csv("rb_proj.csv", index=False)

# NEXT SEASON

In [24]:
qb_ns = adp[(adp.Season == 2023) & (adp.POS.str.contains("QB"))]

In [29]:
rb_ns = adp[(adp.Season == 2023) & (adp.POS.str.contains("RB"))]
wr_ns = adp[(adp.Season == 2023) & (adp.POS.str.contains("WR"))]
te_ns = adp[(adp.Season == 2023) & (adp.POS.str.contains("TE"))]

In [30]:
qb_ns.to_csv("ns data/qb_ns.csv", index=False)
rb_ns.to_csv("ns data/rb_ns.csv", index=False)
wr_ns.to_csv("ns data/wr_ns.csv", index=False)
te_ns.to_csv("ns data/te_ns.csv", index=False) 

In [49]:
rbs_comp[(rbs_comp.pid == 16483) & (rbs_comp.Season != 2023)]["FPTS/G_MISC"].tail(4).mean()

17.075